<a href="https://colab.research.google.com/github/moh2236945/CNNs/blob/master/network_in_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

main Features for NIN:

reduce or increase dimensionality

applu non linearity again after conv

consider as features pooling 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

from torchvision import datasets
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
import math
import time
import random
import copy




class NiN(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.output_dim=output_dim
        self.classifier = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 160, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(160, 96, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.Dropout(0.5),
            # Block 2
            nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
            nn.Dropout(0.5),
            # Block 3
            nn.Conv2d(192, 192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, output_dim, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=8, stride=1, padding=0),
            nn.Dropout(0.5),            
        )
        
    def forward(self, x):
        x = self.classifier(x)
        x = x.view(x.shape[0], self.output_dim)
        prob = torch.softmax(x, dim=1)
        return prob



def num_params(model):
    return sum([params.numel()  for params in model.parameters() if params.requires_grad])

OUTPUT_DIM = 196
model = NiN(OUTPUT_DIM)
def init_weight(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data, nonlinearity = 'relu')
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m,nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))
        nn.init.constant_(m.bias.data, 0)
model.apply(init_weight)
print(model)
print(f"Number of parameters : {num_params(model):,}")

NiN(
  (classifier): Sequential(
    (0): Conv2d(3, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): Conv2d(192, 160, kernel_size=(1, 1), stride=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(160, 96, kernel_size=(1, 1), stride=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.5, inplace=False)
    (8): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): ReLU(inplace=True)
    (10): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (13): ReLU(inplace=True)
    (14): AvgPool2d(kernel_size=3, stride=2, padding=1)
    (15): Dropout(p=0.5, inplace=False)
    (16): Conv2d(192, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (17): ReLU(inplace=True)
    (18): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (19): ReL